In [1]:
import modern_robotics as mr
import numpy as np
from calibration import *
np.set_printoptions(precision=16)

In [2]:
s = 0.001

w1 = np.array([0,0,1])
p1 = np.array([0,0,0])*s

w2 = np.array([0,-1,0])
p2 = np.array([0,0,0])*s

w3 = np.array([0,-1,0])
p3 = np.array([0,0,-100])*s

w4 = np.array([0,0,-1])
p4 = np.array([-50,250,0])*s

w5 = np.array([0,-1,0])
p5 = np.array([-20,0,-250])*s

w6 = np.array([0,0,-1])
p6 = np.array([-50,250,0])*s



s = 0.001
ax1 = mr.ScrewToAxis(p1,w1,0)
ax2 = mr.ScrewToAxis(p2,w2,0)
ax3 = mr.ScrewToAxis(p3,w3,0)
ax4 = mr.ScrewToAxis(p4,w4,0)
ax5 = mr.ScrewToAxis(p5,w5,0)
ax6 = mr.ScrewToAxis(p6,w6,0)

xi0 = np.array([[0, 0, 0, 0, 0, 0],
    [0, -1, -1, 0, -1, 0],
    [1, 0, 0, -1, 0, -1],

    [0, 0, 0, -0.05, -0.02, -0.05],
    [0, 0, 0, 0.25, 0, 0.25],
    [0, 0, -0.1, 0, -0.25, 0]
])



# xi0 = np.c_[ax1, ax2, ax3, ax4, ax5, ax6]

wist0 = np.array([0,0,0])
pist0 = np.array([250,50,-20])*s

# xist0 = mr.ScrewToAxis(wist0, pist0, 0)

one = np.array([0.7868e-4, 0.7445e-4, -0.5380e-4, 0.7445e-4])
two = np.array([-0.0114e-4, -0.8383e-4, 0.8715e-4, 0.7445e-4])
three = np.array([-0.3473e-4, 0.2263e-4, 0.6494e-4, 0.7445e-4])


xist0 = np.r_[wist0, pist0]

xi01=np.array([0.0399999800000150,-0.0199999900000075,0.998999500500375,0.0200000000000000,0.0400000000000000,0])
xi02=np.array([0,-1,0,-0.0200000000000000,0,0.0500000000000000])
xi03=np.array([0.178005251232368,-0.984029029284552,-0.00100002950130544,-0.0841845888907446,0.0874136824072620,-100.999920311298])
xi04=np.array([0.0619994730067192,0.0129998895014089,-0.997991517108157,-50.9999969248523,249.000000644789,0.0751505000414993])
xi05=np.array([0.000999959501660306,-0.999999500040373,0,-20.6000000008239,-0.0205991760337826,-249])
xi06=np.array([0.0949994775043106,0.0309998295014066,-0.994994527545148,-51.2730269967030,248.910906980023,2.85959854441601])

xi00=np.c_[xi01,xi02,xi03,xi04,xi05,xi06]
xi00[3:6,:] = xi00[3:6,:]*s

#initial twist
xist00=np.array([0.02, -0.01, 0.01, 249, 51, -20.6])
xist00[3:6] = xist00[3:6]*s

deq1=0
deq2=0.02
deq3=0.002
deq4=0.02

mean_error_before = np.zeros((10,1))
mean_error_traditional = np.zeros((10,1))

max_error_before = np.zeros((10,1))
max_error_traditional = np.zeros((10,1))


In [3]:
# for N in range(10,110,10):
for N in [10]:

    P01=np.array([100,0,0])*s
    P02=np.array([0,100,0])*s
    P03=np.array([0,0,100])*s
    PX=np.c_[P01,P02,P03]

    P04=np.array([-100,-100,-100])*s

    
    gm=np.zeros((4,4,N))
    gn=np.zeros((4,4,N))
    Pa1=np.zeros((4,N))
    Pa2=np.zeros((4,N))
    Pa3=np.zeros((4,N))
    M_home = np.eye(4)

    cali_joint_angles = np.random.rand(N,6) * 2 * np.pi - np.pi

    # cali_joint_angles = np.array([[-0.7142,   -2.5075,    0.7840,   -1.4789,    2.8647,   -1.6990],
    # [-2.1130,   -2.1558,   -0.8665,   -1.7466,    1.8861,    0.4531],
    # [-0.1401,   -0.1239,   -0.5611,   -2.1584,   -1.0035,    0.8727],
    # [ 2.7880,    0.2781,    2.8878,   -1.7220,   -1.3114,    1.4330],
    # [-1.9297,   -1.8922,   -1.3101,   -1.1336,    1.7620,   -1.0811],
    # [ 2.8279,    2.0362,    0.3295,   -0.7737,   -1.3580,    2.3872],
    # [-1.2938,    1.3721,    0.9346,    3.1137,    1.1232,   -2.1049],
    # [ 2.2178,   -0.7796,    1.7568,   -2.5607,   -0.8643,   -1.4389],
    # [-1.4760,   -1.3966,    2.7994,    1.0531,   -0.4687,   -0.7368],
    # [ 1.0261,    2.0229,   -2.6988,   -2.1006,   -1.0845,    2.5215]])

    for i in range(0,N):
        
        Pa1[:, i] = np.dot(mr.FKinSpace(se_3_exp(xist00),xi00,cali_joint_angles[i,:]), np.r_[P01,1].reshape((4,1)) ).reshape((4,)) + one # ((np.random.rand(4,1)*0.2-0.1) * s).reshape((4,))
        Pa2[:, i] = np.dot(mr.FKinSpace(se_3_exp(xist00),xi00,cali_joint_angles[i,:]), np.r_[P02,1].reshape((4,1)) ).reshape((4,)) + two # ((np.random.rand(4,1)*0.2-0.1) * s).reshape((4,))
        Pa3[:, i] = np.dot(mr.FKinSpace(se_3_exp(xist00),xi00,cali_joint_angles[i,:]), np.r_[P03,1].reshape((4,1)) ).reshape((4,)) + three # ((np.random.rand(4,1)*0.2-0.1) * s).reshape((4,))
        
        PY = np.c_[Pa1[0:3,i], Pa2[0:3,i], Pa3[0:3,i]]

        R,t = registration(PX, PY)

        gm[0:3,0:3,i] = R
        gm[0:3,3,i] = t
        gn[:,:,i] = mr.FKinSpace(se_3_exp(xist00),xi00,cali_joint_angles[i,:])

    xiTFP, xistTFP = traditional_calibration_6dof(xi0, xist00, cali_joint_angles, gm, 10)

    Nt=40
    
    error_before=np.zeros((Nt,1))
        
    error_afterTraditional=np.zeros((Nt,1))
    
    test_joint_config = np.c_[np.random.rand(Nt,1) * 2 * np.pi,
                            np.random.rand(Nt,1) * 2 * np.pi, 
                            np.random.rand(Nt,1) * 2 * np.pi, 
                            np.random.rand(Nt,1) * 2 * np.pi, 
                            np.random.rand(Nt,1) * 2 * np.pi, 
                            np.random.rand(Nt,1) * 2 * np.pi]

    # test_joint_config = np.array([[1.8545,    3.9109,    4.5162,    6.2266,    5.5974,    5.9742],
    # [1.4847,    0.4230,    2.3975,    4.7979,    3.3604,    4.9235],
    # [0.7878,    5.5578,    2.0520,    5.9159,    3.0841,    5.9240],
    # [3.9972,    1.7798,    6.0215,    0.5746,    6.2299,    3.2293],
    # [4.6179,    0.1013,    0.9226,    5.5751,    1.1184,    3.3958],
    # [4.9933,    4.7473,    2.8289,    0.8647,    1.3867,    2.7265],
    # [0.9814,    1.8176,    1.4647,    2.6019,    4.4786,    2.4053],
    # [1.4173,    3.8486,    3.4075,    3.4840,    3.2156,    3.3476],
    # [3.0615,    1.1494,    2.5016,    2.2700,    3.7842,    3.5913],
    # [0.0442,    1.2921,    6.0238,    5.1130,    4.0562,    1.6948],
    # [5.3600,    2.3882,    2.2918,    6.0175,    2.7266,    3.6662],
    # [4.3135,    1.2420,    3.6287,    4.5580,    3.0095,    5.0368],
    # [3.9722,    2.7629,    4.4884,    1.9943,    6.2810,    3.7601],
    # [2.8265,    4.8912,    2.5853,    2.9092,    3.4039,    2.9045],
    # [5.7010,    5.8913,    4.9070,    1.4587,    0.2363,    3.9525],
    # [4.6703,    3.1395,    2.4066,    2.1767,    0.5518,    3.0839],
    # [0.8954,    3.9558,    0.4345,    1.3127,    3.8217,    2.6344],
    # [1.3363,    2.9034,    3.4200,    5.6542,    5.5115,    0.0238],
    # [5.6621,    1.0549,    2.7003,    2.0935,    0.7933,    1.5582],
    # [1.1085,    2.9062,    2.6148,    2.0260,    1.0224,    6.2119],
    # [4.2249,    4.6731,    6.2120,    5.1011,    5.9489,    5.0050],
    # [6.2365,    0.0107,    5.7866,    0.4336,    3.1404,    0.5236],
    # [2.5466,    4.8401,    3.5010,    2.5512,    2.5638,    5.3936],
    # [4.8035,    5.1382,    5.3548,    3.5266,    3.2568,    3.6998],
    # [5.9889,    3.8213,    2.0271,    2.1548,    5.5418,    0.7483],
    # [0.8885,    0.6543,    3.5466,    4.5363,    2.7053,    4.3189],
    # [1.7077,    0.9121,    3.0537,    1.1782,    1.3384,    5.1697],
    # [6.2445,    3.0210,    4.9331,    2.5280,    5.9238,    5.9830],
    # [5.8915,    2.6798,    3.9116,    0.4843,    1.5172,    1.4928],
    # [2.1507,    1.4839,    3.5110,    2.8867,    1.8646,    0.7910],
    # [2.6565,    0.1779,    1.1852,    5.8674,    2.6255,    5.5494],
    # [0.7018,    1.7681,    2.2859,    2.1149,    4.5863,    4.7047],
    # [4.5972,    4.6752,    0.7278,    6.0068,    1.6957,    1.5863],
    # [5.8697,    4.6742,    6.0143,    2.9620,    3.1264,    5.4675],
    # [4.6141,    5.5829,    2.2767,    1.1189,    3.2683,    2.2087],
    # [1.5710,    1.2720,    1.2980,    3.4225,    3.9183,    2.3579],
    # [1.9170,    2.9074,    0.7114,    3.9203,    0.8213,    1.2446],
    # [2.2390,    0.4703,    3.8480,    3.1990,    2.0283,    5.1119],
    # [3.1104,    2.7105,    1.4173,    3.0492,    1.4136,    6.2360],
    # [3.6834,    1.1383,    3.5483,    2.8726,    3.5566,    5.7939]])


    for i in range(0,Nt):
        
        error_before[i] = np.linalg.norm( np.dot( mr.FKinSpace(se_3_exp(xist0),xi0, test_joint_config[i,:]) - mr.FKinSpace(se_3_exp(xist00),np.round(xi00,4),test_joint_config[i,:]), np.r_[P04,1])) / s
        
        error_afterTraditional[i] = np.linalg.norm( np.dot( mr.FKinSpace(se_3_exp(xistTFP), np.round(xiTFP,4), np.round(test_joint_config[i,:],4)) - mr.FKinSpace(se_3_exp(xist00),np.round(xi00,4), np.round(test_joint_config[i,:],4)), np.r_[P04,1]) ) / s

    idx = int(N/10) - 1

    mean_error_before[idx, 0] = np.mean(error_before)
    mean_error_traditional[idx, 0] = np.mean(error_afterTraditional)


In [12]:
from datetime import date
print(date.today())
np.savetxt("Test_{}".format(date.today()),xi0)

2023-01-27


In [5]:
print("Mean error before: ", mean_error_before)
print("Mean error traditional: ", mean_error_traditional)

Mean error before:  [[50.779045798633355]
 [ 0.               ]
 [ 0.               ]
 [ 0.               ]
 [ 0.               ]
 [ 0.               ]
 [ 0.               ]
 [ 0.               ]
 [ 0.               ]
 [ 0.               ]]
Mean error traditional:  [[0.3865094035693736]
 [0.                ]
 [0.                ]
 [0.                ]
 [0.                ]
 [0.                ]
 [0.                ]
 [0.                ]
 [0.                ]
 [0.                ]]


In [1]:
inp = input()

In [3]:
inp == '1'

True